In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df = pd.read_csv("smart_maintenance_dataset.csv")

df.head()


,Machine_ID,Temp_C,Vibration_mm_s,Pressure_Bar,Acoustic_dB,Inspection_Duration_min,Downtime_Cost_USD,Technician_Availability_pct,Failure_Prob,Maintenance_Priority
0,M004,70.415631,3.036378,5.582562,74.494678,68,3112,99,0.48,2
1,M006,74.509666,2.787866,4.916866,64.017351,127,522,63,0.50,2
2,M009,78.724957,2.067099,4.597601,68.772382,68,836,84,0.47,2
3,M006,76.677258,1.998635,5.012083,76.909872,142,3799,80,0.48,2
4,M009,53.381545,3.844676,5.176345,84.855886,88,2710,78,0.51,2


In [4]:
df.shape

(1430, 10)

In [5]:
df.columns

Index(['Machine_ID', 'Temp_C', 'Vibration_mm_s', 'Pressure_Bar', 'Acoustic_dB',
       'Inspection_Duration_min', 'Downtime_Cost_USD',
       'Technician_Availability_pct', 'Failure_Prob', 'Maintenance_Priority'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1430 entries, 0 to 1429
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Machine_ID                   1430 non-null   object 
 1   Temp_C                       1430 non-null   float64
 2   Vibration_mm_s               1430 non-null   float64
 3   Pressure_Bar                 1430 non-null   float64
 4   Acoustic_dB                  1430 non-null   float64
 5   Inspection_Duration_min      1430 non-null   int64  
 6   Downtime_Cost_USD            1430 non-null   int64  
 7   Technician_Availability_pct  1430 non-null   int64  
 8   Failure_Prob                 1430 non-null   float64
 9   Maintenance_Priority         1430 non-null   int64  
dtypes: float64(5), int64(4), object(1)
memory usage: 111.8+ KB


In [7]:
df.isnull().sum()


Machine_ID                     0
Temp_C                         0
Vibration_mm_s                 0
Pressure_Bar                   0
Acoustic_dB                    0
Inspection_Duration_min        0
Downtime_Cost_USD              0
Technician_Availability_pct    0
Failure_Prob                   0
Maintenance_Priority           0
dtype: int64

In [8]:
print("Duplicate rows:", df.duplicated().sum())
df = df.drop_duplicates()



Duplicate rows: 0


In [9]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1430 entries, 0 to 1429
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Machine_ID                   1430 non-null   object 
 1   Temp_C                       1430 non-null   float64
 2   Vibration_mm_s               1430 non-null   float64
 3   Pressure_Bar                 1430 non-null   float64
 4   Acoustic_dB                  1430 non-null   float64
 5   Inspection_Duration_min      1430 non-null   int64  
 6   Downtime_Cost_USD            1430 non-null   int64  
 7   Technician_Availability_pct  1430 non-null   int64  
 8   Failure_Prob                 1430 non-null   float64
 9   Maintenance_Priority         1430 non-null   int64  
dtypes: float64(5), int64(4), object(1)
memory usage: 111.8+ KB


In [10]:
# Check unique values in the Maintenance_Priority column
unique_values = df["Maintenance_Priority"].unique()
print("Unique values:", unique_values)

# Check if there are any values other than 1, 2, and 3
expected_values = {1, 2, 3}
actual_values = set(unique_values)

unexpected_values = actual_values - expected_values

if unexpected_values:
    print("Unexpected values found:", unexpected_values)
else:
    print("Only expected values (1, 2, 3) are present.")

Unique values: [2 1 3]
Only expected values (1, 2, 3) are present.


In [11]:
from sklearn.model_selection import train_test_split

# Define input features (X) and target (y)
X = df[['Temp_C', 'Vibration_mm_s', 'Pressure_Bar',
        'Inspection_Duration_min', 'Downtime_Cost_USD',
        'Technician_Availability_pct']]

y = df['Maintenance_Priority']

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Check shapes
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test :", X_test.shape)
print("Shape of y_test :", y_test.shape)


Shape of X_train: (1144, 6)
Shape of y_train: (1144,)
Shape of X_test : (286, 6)
Shape of y_test : (286,)


In [12]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)

X_train_scaled = sc.transform(X_train)
X_test_scaled = sc.transform(X_test)


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize and train the model
model = LogisticRegression(random_state=42, multi_class='multinomial', solver='lbfgs', max_iter=1000)
model.fit(X_train_scaled, y_train)

# Predict
y_pred = model.predict(X_test_scaled)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.972027972027972

Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00        60
           2       0.97      1.00      0.98       214
           3       0.83      0.42      0.56        12

    accuracy                           0.97       286
   macro avg       0.93      0.80      0.85       286
weighted avg       0.97      0.97      0.97       286


Confusion Matrix:
 [[ 60   0   0]
 [  0 213   1]
 [  0   7   5]]


C:\Users\karti\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [14]:
def predict():
    # take data from form and store in each feature    
    #input_features = [x for x in request.form.values()]  # ❌ this line is Flask-based, not for Tkinter (remove if using Tkinter)
    
    # ✅ Replace these with your Smart Maintenance features
    temperature = input_features[0]
    vibration = input_features[1]
    pressure = input_features[2]
    inspection_time = input_features[3]
    downtime_cost = input_features[4]
    technician_availability = input_features[5]

    # ✅ Predict the maintenance priority by calling your model's prediction function
    predicted_priority = model.predict_maintenance_priority(
        temperature, vibration, pressure, inspection_time, downtime_cost, technician_availability
    )


In [15]:
import joblib

# Assuming your model is named 'model' and your scaler is named 'scaler'
joblib.dump(model, 'model.pkl')
joblib.dump(sc, 'scaler.pkl')


['scaler.pkl']

In [16]:
import joblib

# Load the trained model and scaler
model = joblib.load('model.pkl')
scaler = joblib.load('scaler.pkl')


In [17]:
import numpy as np

def predict_machine_condition(model, scaler, temperature, vibration, sound, load, shift, operator_id):
    # Create a zero NumPy array with number of columns = total features after encoding
    x = np.zeros(len(X.columns))

    # Fill in the values based on feature index
    x[0] = temperature
    x[1] = vibration
    x[2] = sound
    x[3] = load

    # Handle one-hot encoding for categorical features
    if 'shift_' + shift in X.columns:
        shift_index = np.where(X.columns == 'shift_' + shift)[0][0]
        x[shift_index] = 1

    if 'operator_' + operator_id in X.columns:
        operator_index = np.where(X.columns == 'operator_' + operator_id)[0][0]
        x[operator_index] = 1

    # Feature scaling
    x_scaled = scaler.transform([x])[0]

    # Predict using model
    return model.predict([x_scaled])[0]


In [18]:
#GUI TKINTER CODE STARTS FROM HERE 

In [19]:
'''import tkinter as tk
from tkinter import messagebox
import joblib
import numpy as np

# Load model and scaler
model = joblib.load('model.pkl')
scaler = joblib.load('scaler.pkl')

def predict_maintenance_priority():
    try:
        # Collect input data
        temp = float(entry_temp.get())
        vibration = float(entry_vibration.get())
        pressure = float(entry_pressure.get())
        inspection_time = float(entry_inspection.get())
        downtime_cost = float(entry_downtime.get())
        technician_availability = float(entry_technician.get())

        # Prepare input array
        input_data = np.array([[temp, vibration, pressure, inspection_time, downtime_cost, technician_availability]])
        input_scaled = scaler.transform(input_data)

        # Predict
        prediction = model.predict(input_scaled)[0]
        messagebox.showinfo("Prediction", f"Predicted Maintenance Priority: {prediction}")

    except Exception as e:
        messagebox.showerror("Error", f"Invalid input: {e}")

# Create GUI
root = tk.Tk()
root.title("Smart Maintenance Predictor")
root.geometry("400x500")

font_label = ("Arial", 12)
font_entry = ("Arial", 12)

labels = [
    ("Temperature (C):", "temp"),
    ("Vibration (mm/s):", "vibration"),
    ("Pressure (Bar):", "pressure"),
    ("Inspection Duration (min):", "inspection"),
    ("Downtime Cost (USD):", "downtime"),
    ("Technician Availability (%):", "technician")
]

entries = {}

for label_text, var_name in labels:
    label = tk.Label(root, text=label_text, font=font_label)
    label.pack(pady=5)
    entry = tk.Entry(root, font=font_entry)
    entry.pack(pady=5)
    entries[var_name] = entry

entry_temp = entries['temp']
entry_vibration = entries['vibration']
entry_pressure = entries['pressure']
entry_inspection = entries['inspection']
entry_downtime = entries['downtime']
entry_technician = entries['technician']

predict_button = tk.Button(root, text="Predict Maintenance Priority", font=("Arial", 14), command=predict_maintenance_priority)
predict_button.pack(pady=20)

exit_button = tk.Button(root, text="Exit", font=("Arial", 12), command=root.destroy)
exit_button.pack(pady=10)

root.mainloop()'''


'import tkinter as tk\nfrom tkinter import messagebox\nimport joblib\nimport numpy as np\n\n# Load model and scaler\nmodel = joblib.load(\'model.pkl\')\nscaler = joblib.load(\'scaler.pkl\')\n\ndef predict_maintenance_priority():\n    try:\n        # Collect input data\n        temp = float(entry_temp.get())\n        vibration = float(entry_vibration.get())\n        pressure = float(entry_pressure.get())\n        inspection_time = float(entry_inspection.get())\n        downtime_cost = float(entry_downtime.get())\n        technician_availability = float(entry_technician.get())\n\n        # Prepare input array\n        input_data = np.array([[temp, vibration, pressure, inspection_time, downtime_cost, technician_availability]])\n        input_scaled = scaler.transform(input_data)\n\n        # Predict\n        prediction = model.predict(input_scaled)[0]\n        messagebox.showinfo("Prediction", f"Predicted Maintenance Priority: {prediction}")\n\n    except Exception as e:\n        messageb

In [ ]:
#CLAUDE AI CODE 

#FULLY WORKING CODE 

#GUI


import tkinter as tk
from tkinter import messagebox, ttk
import joblib
import numpy as np
from datetime import datetime
import threading
import time

# Load model and scaler
try:
    model = joblib.load('model.pkl')
    scaler = joblib.load('scaler.pkl')
    MODEL_LOADED = True
except:
    MODEL_LOADED = False
    print("Warning: Model files not found. Using mock prediction for demo.")

class ScrollableFrame(tk.Frame):
    """A scrollable frame that can contain other widgets"""
    def __init__(self, container, *args, **kwargs):
        super().__init__(container, *args, **kwargs)
        
        # Create canvas and scrollbar
        self.canvas = tk.Canvas(self, bg='#0f1419', highlightthickness=0)
        self.scrollbar = ttk.Scrollbar(self, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = tk.Frame(self.canvas, bg='#0f1419')
        
        # Configure scrolling
        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all"))
        )
        
        # Create window in canvas
        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)
        
        # Pack canvas and scrollbar
        self.canvas.pack(side="left", fill="both", expand=True)
        self.scrollbar.pack(side="right", fill="y")
        
        # Bind mouse wheel to canvas
        self.canvas.bind("<MouseWheel>", self._on_mousewheel)
        self.canvas.bind("<Button-4>", self._on_mousewheel)
        self.canvas.bind("<Button-5>", self._on_mousewheel)
        
        # Bind mouse wheel to scrollable frame and all its children
        self.scrollable_frame.bind("<MouseWheel>", self._on_mousewheel)
        self._bind_to_mousewheel(self.scrollable_frame)
        
    def _bind_to_mousewheel(self, widget):
        """Bind mouse wheel to widget and all its children"""
        widget.bind("<MouseWheel>", self._on_mousewheel)
        for child in widget.winfo_children():
            self._bind_to_mousewheel(child)
    
    def _on_mousewheel(self, event):
        """Handle mouse wheel scrolling"""
        # For Windows and MacOS
        if event.delta:
            self.canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")
        # For Linux
        elif event.num == 4:
            self.canvas.yview_scroll(-1, "units")
        elif event.num == 5:
            self.canvas.yview_scroll(1, "units")

class ModernGUI:
    def __init__(self, root):
        self.root = root
        self.setup_window()
        self.setup_styles()
        self.create_widgets()
        self.animate_title()
        
    def setup_window(self):
        """Configure the main window"""
        self.root.title("🔧 Smart Maintenance Predictor")
        self.root.geometry("900x800")
        self.root.resizable(True, True)
        
        # Configure window background
        self.root.configure(bg='#0f1419')
        
        # Center the window on screen
        self.root.update_idletasks()
        width = self.root.winfo_width()
        height = self.root.winfo_height()
        x = (self.root.winfo_screenwidth() // 2) - (width // 2)
        y = (self.root.winfo_screenheight() // 2) - (height // 2)
        self.root.geometry(f'{width}x{height}+{x}+{y}')
        
    def setup_styles(self):
        """Configure modern styling"""
        style = ttk.Style()
        style.theme_use('clam')
        
        # Configure scrollbar style
        style.configure('Vertical.TScrollbar',
                       background='#233554',
                       troughcolor='#1a1f2e',
                       borderwidth=0,
                       arrowcolor='#00d4ff',
                       darkcolor='#233554',
                       lightcolor='#233554')
        
    def create_widgets(self):
        """Create all GUI widgets with scrollable interface"""
        # Create main scrollable frame
        self.scroll_frame = ScrollableFrame(self.root)
        self.scroll_frame.pack(fill='both', expand=True, padx=20, pady=20)
        
        # Main content goes into the scrollable frame
        main_frame = self.scroll_frame.scrollable_frame
        
        # Header section
        self.create_header(main_frame)
        
        # Input fields section
        self.create_input_section(main_frame)
        
        # Buttons section
        self.create_button_section(main_frame)
        
        # Status bar (fixed at bottom)
        self.create_status_bar()
        
    def create_header(self, parent):
        """Create header with title and subtitle"""
        header_frame = tk.Frame(parent, bg='#0f1419')
        header_frame.pack(fill='x', pady=(20, 40))
        
        # Animated title
        self.title_label = tk.Label(header_frame, 
                                   text="🔧 Smart Maintenance Predictor",
                                   font=('Segoe UI', 32, 'bold'),
                                   fg='#00d4ff',
                                   bg='#0f1419')
        self.title_label.pack(pady=(0, 15))
        
        # Subtitle with current time
        current_time = datetime.now().strftime("%B %d, %Y • %I:%M %p")
        subtitle_text = f"AI-Powered Predictive Maintenance System • {current_time}"
        
        self.subtitle_label = tk.Label(header_frame,
                                      text=subtitle_text,
                                      font=('Segoe UI', 13),
                                      fg='#8892b0',
                                      bg='#0f1419')
        self.subtitle_label.pack()
        
        # Decorative line
        line_frame = tk.Frame(header_frame, height=3, bg='#00d4ff')
        line_frame.pack(fill='x', pady=(20, 0))
        
        # Scroll instruction
        instruction_label = tk.Label(header_frame,
                                   text="💡 Scroll down to access all fields or use mouse wheel",
                                   font=('Segoe UI', 10, 'italic'),
                                   fg='#64ffda',
                                   bg='#0f1419')
        instruction_label.pack(pady=(10, 0))
        
    def create_input_section(self, parent):
        """Create input fields with proper spacing and scrollable design"""
        # Container for input fields
        input_container = tk.Frame(parent, bg='#1a1f2e', relief='flat', bd=0)
        input_container.pack(fill='x', pady=(20, 30))
        
        # Add subtle border effect
        border_frame = tk.Frame(input_container, bg='#2d3748', height=2)
        border_frame.pack(fill='x', side='top')
        
        # Title for input section
        section_title = tk.Label(input_container, 
                                text="📊 Equipment Parameters",
                                font=('Segoe UI', 20, 'bold'),
                                fg='#00d4ff',
                                bg='#1a1f2e')
        section_title.pack(pady=(30, 25))
        
        # Input fields configuration
        self.fields = [
            ("🌡️ Temperature (°C)", "temp", "Enter temperature value", "25.5"),
            ("📳 Vibration (mm/s)", "vibration", "Enter vibration level", "0.8"),
            ("⚡ Pressure (Bar)", "pressure", "Enter pressure reading", "15.2"),
            ("⏱️ Inspection Duration (min)", "inspection", "Enter inspection time", "45"),
            ("💰 Downtime Cost (USD)", "downtime", "Enter estimated cost", "1500"),
            ("👥 Technician Availability (%)", "technician", "Enter availability percentage", "85")
        ]
        
        # Create input fields with generous spacing
        self.entries = {}
        fields_frame = tk.Frame(input_container, bg='#1a1f2e')
        fields_frame.pack(fill='x', padx=40, pady=(0, 30))
        
        for i, (label_text, var_name, placeholder, example) in enumerate(self.fields):
            # Create field container with more space
            field_frame = tk.Frame(fields_frame, bg='#1a1f2e')
            field_frame.pack(fill='x', pady=15)  # Increased padding
            
            # Label with better styling
            label = tk.Label(field_frame, 
                           text=label_text, 
                           font=('Segoe UI', 13, 'bold'),
                           fg='#ccd6f6',
                           bg='#1a1f2e')
            label.pack(anchor='w', pady=(0, 8))
            
            # Entry with more height and better styling
            entry = tk.Entry(field_frame, 
                           font=('Segoe UI', 12),
                           bg='#233554',
                           fg='#ccd6f6',
                           insertbackground='#00d4ff',
                           bd=0,
                           relief='flat',
                           highlightthickness=2,
                           highlightbackground='#2d3748',
                           highlightcolor='#00d4ff')
            entry.pack(fill='x', ipady=12)  # Increased internal padding
            
            # Store the entry FIRST
            self.entries[var_name] = entry
            
            # Add placeholder text
            entry.insert(0, f"e.g., {example}")
            entry.config(fg='#64748b')
            
            # Bind events for placeholder effect
            def make_handlers(entry_widget, example_text, var_name):
                return (
                    lambda e: self.on_entry_focus_in(e, entry_widget, example_text, var_name),
                    lambda e: self.on_entry_focus_out(e, entry_widget, example_text, var_name),
                    lambda e: self.on_entry_key_press(e, entry_widget, example_text, var_name)
                )
            
            focus_in_handler, focus_out_handler, key_press_handler = make_handlers(entry, example, var_name)
            
            entry.bind('<FocusIn>', focus_in_handler)
            entry.bind('<FocusOut>', focus_out_handler)
            entry.bind('<KeyPress>', key_press_handler)
            
            # Add field separator line except for last field
            if i < len(self.fields) - 1:
                separator = tk.Frame(field_frame, height=1, bg='#2d3748')
                separator.pack(fill='x', pady=(15, 0))
                
    def create_button_section(self, parent):
        """Create buttons with proper spacing"""
        button_frame = tk.Frame(parent, bg='#0f1419')
        button_frame.pack(fill='x', pady=(20, 40))
        
        # Predict button with loading animation capability
        self.predict_button = tk.Button(button_frame,
                                      text="🔮 Predict Maintenance Priority",
                                      font=('Segoe UI', 16, 'bold'),
                                      bg='#00d4ff',
                                      fg='#0f1419',
                                      activebackground='#0099cc',
                                      activeforeground='#ffffff',
                                      bd=0,
                                      relief='flat',
                                      cursor='hand2',
                                      pady=18,
                                      command=self.predict_maintenance_priority)
        self.predict_button.pack(fill='x', pady=(0, 15))
        
        # Button effects
        self.predict_button.bind('<Enter>', lambda e: self.button_hover(e, '#0099cc'))
        self.predict_button.bind('<Leave>', lambda e: self.button_leave(e, '#00d4ff'))
        
        # Secondary buttons frame
        secondary_frame = tk.Frame(button_frame, bg='#0f1419')
        secondary_frame.pack(fill='x')
        
        # Clear button
        clear_button = tk.Button(secondary_frame,
                               text="🗑️ Clear All Fields",
                               font=('Segoe UI', 12),
                               bg='#2d3748',
                               fg='#ccd6f6',
                               activebackground='#4a5568',
                               bd=0,
                               relief='flat',
                               cursor='hand2',
                               pady=12,
                               command=self.clear_all_fields)
        clear_button.pack(side='left', fill='x', expand=True, padx=(0, 8))
        
        # Scroll to top button
        scroll_top_button = tk.Button(secondary_frame,
                                    text="⬆️ Scroll to Top",
                                    font=('Segoe UI', 12),
                                    bg='#2d3748',
                                    fg='#64ffda',
                                    activebackground='#4a5568',
                                    bd=0,
                                    relief='flat',
                                    cursor='hand2',
                                    pady=12,
                                    command=self.scroll_to_top)
        scroll_top_button.pack(side='left', fill='x', expand=True, padx=(4, 4))
        
        # Exit button
        exit_button = tk.Button(secondary_frame,
                              text="❌ Exit Application",
                              font=('Segoe UI', 12),
                              bg='#2d3748',
                              fg='#ff6b6b',
                              activebackground='#ff8a8a',
                              activeforeground='#ffffff',
                              bd=0,
                              relief='flat',
                              cursor='hand2',
                              pady=12,
                              command=self.root.destroy)
        exit_button.pack(side='right', fill='x', expand=True, padx=(8, 0))
        
    def create_status_bar(self):
        """Create fixed status bar at the bottom"""
        # Create status bar frame that stays at the bottom
        status_container = tk.Frame(self.root, bg='#1a1f2e', height=40)
        status_container.pack(fill='x', side='bottom')
        status_container.pack_propagate(False)  # Prevent shrinking
        
        # Status content
        status_frame = tk.Frame(status_container, bg='#1a1f2e')
        status_frame.pack(fill='both', expand=True, padx=20, pady=8)
        
        # Status indicator
        model_status = "✅ Model Ready" if MODEL_LOADED else "⚠️ Demo Mode"
        self.status_label = tk.Label(status_frame,
                                   text=f"{model_status} • Accuracy: 97% • Last Updated: {datetime.now().strftime('%H:%M:%S')}",
                                   font=('Segoe UI', 10),
                                   fg='#64ffda',
                                   bg='#1a1f2e')
        self.status_label.pack(side='left')
        
        # Scroll position indicator
        self.scroll_indicator = tk.Label(status_frame,
                                       text="📍 Scroll Position: Top",
                                       font=('Segoe UI', 10),
                                       fg='#8892b0',
                                       bg='#1a1f2e')
        self.scroll_indicator.pack(side='right')
        
    def scroll_to_top(self):
        """Scroll to the top of the form"""
        self.scroll_frame.canvas.yview_moveto(0)
        self.scroll_indicator.config(text="📍 Scroll Position: Top")
        
    def on_entry_focus_in(self, event, entry, example, var_name):
        """Handle entry focus in (remove placeholder)"""
        if entry.get() == f"e.g., {example}" or entry.cget('fg') == '#64748b':
            entry.delete(0, tk.END)
            entry.config(fg='#ccd6f6')
            
    def on_entry_focus_out(self, event, entry, example, var_name):
        """Handle entry focus out (add placeholder if empty)"""
        if not entry.get() or entry.get().strip() == "":
            entry.delete(0, tk.END)
            entry.insert(0, f"e.g., {example}")
            entry.config(fg='#64748b')
            
    def on_entry_key_press(self, event, entry, example, var_name):
        """Handle key press to clear placeholder immediately when typing"""
        if entry.cget('fg') == '#64748b':  # If placeholder is showing
            entry.delete(0, tk.END)
            entry.config(fg='#ccd6f6')
            
    def button_hover(self, event, color):
        """Button hover effect"""
        event.widget.config(bg=color)
        
    def button_leave(self, event, color):
        """Button leave effect"""
        event.widget.config(bg=color)
        
    def clear_all_fields(self):
        """Clear all input fields"""
        for var_name, entry in self.entries.items():
            entry.delete(0, tk.END)
            entry.config(fg='#ccd6f6')
            # Get the example value for this field
            for label_text, field_var, placeholder, example in self.fields:
                if field_var == var_name:
                    entry.insert(0, f"e.g., {example}")
                    entry.config(fg='#64748b')
                    break
                    
    def animate_title(self):
        """Animate the title color"""
        colors = ['#00d4ff', '#64ffda', '#ff6b6b', '#ffd700', '#00d4ff']
        
        def animate():
            for color in colors:
                self.title_label.config(fg=color)
                self.root.update()
                time.sleep(0.8)
                
        # Run animation in a separate thread
        animation_thread = threading.Thread(target=animate, daemon=True)
        animation_thread.start()
        
    def show_loading(self):
        """Show loading state"""
        self.predict_button.config(text="🔄 Processing...", state='disabled')
        self.root.update()
        
    def hide_loading(self):
        """Hide loading state"""
        self.predict_button.config(text="🔮 Predict Maintenance Priority", state='normal')
        
    def predict_maintenance_priority(self):
        """Predict maintenance priority with enhanced UI feedback"""
        try:
            self.show_loading()
            
            # Collect input data
            input_values = {}
            for var_name, entry in self.entries.items():
                value = entry.get().strip()
                # Skip if placeholder text or empty
                if value.startswith("e.g.,") or value == "" or entry.cget('fg') == '#64748b':
                    field_name = next((label for label, var, _, _ in self.fields if var == var_name), var_name)
                    raise ValueError(f"Please enter a value for {field_name}")
                input_values[var_name] = float(value)
            
            # Prepare input array
            input_data = np.array([[
                input_values['temp'],
                input_values['vibration'],
                input_values['pressure'],
                input_values['inspection'],
                input_values['downtime'],
                input_values['technician']
            ]])
            
            # Simulate processing time for better UX
            time.sleep(1)
            
            if MODEL_LOADED:
                input_scaled = scaler.transform(input_data)
                prediction = model.predict(input_scaled)[0]
            else:
                # Mock prediction for demo
                prediction = np.random.choice(['Low', 'Medium', 'High'])
            
            self.hide_loading()
            
            # Show result with enhanced dialog
            self.show_prediction_result(prediction, input_values)
            
        except ValueError as e:
            self.hide_loading()
            self.show_error("Input Error", str(e))
        except Exception as e:
            self.hide_loading()
            self.show_error("Error", f"An unexpected error occurred: {e}")
            
    def show_prediction_result(self, prediction, input_values):
        """Show prediction result with enhanced formatting"""
        # Create custom result dialog
        result_window = tk.Toplevel(self.root)
        result_window.title("Prediction Result")
        result_window.geometry("550x500")
        result_window.configure(bg='#0f1419')
        result_window.resizable(False, False)
        
        # Center the result window
        result_window.transient(self.root)
        result_window.grab_set()
        
        # Result content
        content_frame = tk.Frame(result_window, bg='#0f1419', padx=30, pady=20)
        content_frame.pack(fill='both', expand=True)
        
        # Title
        title_label = tk.Label(content_frame,
                             text="🎯 Prediction Result",
                             font=('Segoe UI', 22, 'bold'),
                             fg='#00d4ff',
                             bg='#0f1419')
        title_label.pack(pady=(0, 20))
        
        # Priority result
        priority_colors = {'Low': '#64ffda', 'Medium': '#ffd700', 'High': '#ff6b6b'}
        priority_color = priority_colors.get(prediction, '#ccd6f6')
        
        priority_label = tk.Label(content_frame,
                                text=f"Maintenance Priority: {prediction}",
                                font=('Segoe UI', 18, 'bold'),
                                fg=priority_color,
                                bg='#0f1419')
        priority_label.pack(pady=(0, 25))
        
        # Input summary
        summary_frame = tk.Frame(content_frame, bg='#1a1f2e')
        summary_frame.pack(fill='x', pady=(0, 20))
        
        summary_title = tk.Label(summary_frame,
                               text="📋 Input Summary",
                               font=('Segoe UI', 14, 'bold'),
                               fg='#ccd6f6',
                               bg='#1a1f2e')
        summary_title.pack(pady=12)
        
        # Display input values
        for label_text, var_name, _, _ in self.fields:
            value = input_values.get(var_name, 'N/A')
            summary_text = f"{label_text}: {value}"
            summary_label = tk.Label(summary_frame,
                                   text=summary_text,
                                   font=('Segoe UI', 11),
                                   fg='#8892b0',
                                   bg='#1a1f2e')
            summary_label.pack(anchor='w', padx=20, pady=3)
        
        # Close button
        close_button = tk.Button(content_frame,
                               text="✅ Close",
                               font=('Segoe UI', 14, 'bold'),
                               bg='#00d4ff',
                               fg='#0f1419',
                               bd=0,
                               relief='flat',
                               cursor='hand2',
                               pady=12,
                               command=result_window.destroy)
        close_button.pack(fill='x', pady=(15, 0))
        
    def show_error(self, title, message):
        """Show error with custom styling"""
        messagebox.showerror(title, message)
        
    def update_status(self):
        """Update status bar with current time"""
        current_time = datetime.now().strftime('%H:%M:%S')
        model_status = "✅ Model Ready" if MODEL_LOADED else "⚠️ Demo Mode"
        self.status_label.config(text=f"{model_status} • Accuracy: 97% • Last Updated: {current_time}")
        
        # Update scroll position indicator
        try:
            scroll_pos = self.scroll_frame.canvas.canvasy(0)
            max_scroll = self.scroll_frame.canvas.bbox("all")[3]
            if scroll_pos <= 10:
                pos_text = "Top"
            elif scroll_pos >= max_scroll - 500:
                pos_text = "Bottom"
            else:
                pos_text = "Middle"
            self.scroll_indicator.config(text=f"📍 Scroll Position: {pos_text}")
        except:
            pass
            
        # Schedule next update
        self.root.after(1000, self.update_status)

# Create and run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = ModernGUI(root)
    
    # Start status updates
    app.update_status()
    
    root.mainloop()

In [ ]:
import pickle

with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)
